In [29]:
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
import torch
from torch import nn

In [30]:
#决策树回归
model_DT = tree.DecisionTreeRegressor()
#线性回归
model_Line = linear_model.LinearRegression()
#SVM回归
model_SVR = svm.SVR()
#knn回归
model_KNN = neighbors.KNeighborsRegressor()
#随机森林回归
model_RFR = ensemble.RandomForestRegressor()
#Adaboot回归
model_ABR = ensemble.AdaBoostRegressor()
#GBRT回归
model_GBR = ensemble.GradientBoostingRegressor()
#Bagging回归
model_BR = ensemble.BaggingRegressor()
#ExtraTree极端随机树回归
model_ETR = tree.ExtraTreeRegressor()

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
loss = nn.MSELoss()
# def rmse(actual,predict):
#     predict = np.array(predict)
#     actual = np.array(actual)
#
#     distance = predict - actual
#
#     square_distance = distance ** 2
#
#     mean_square_distance = square_distance.mean()
#
#     score = np.sqrt(mean_square_distance)
#
#     return score
def rmse(actual, predict):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds = torch.clamp(predict, 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds),
                           torch.log(actual)))
    return rmse.item()

rmse_score = make_scorer(rmse,greater_is_better=False)

In [35]:
def get_best_model_and_accuracy(model, params, X, y):
    grid = GridSearchCV(model, # 要搜索的模型
                        params, # 要尝试的参数
                        scoring='neg_mean_squared_log_error',
                        error_score=0.) # 如果报错，结果是0
    grid.fit(X, y) # 拟合模型和参数
    # 经典的性能指标
    print("Best Accuracy: {}".format(grid.best_score_))
    # 得到最佳准确率的最佳参数
    print("Best Parameters: {}".format(grid.best_params_))
    # 拟合的平均时间（秒）
    print("Average Time to Fit (s): {}".format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    # 预测的平均时间（秒）
    # 从该指标可以看出模型在真实世界的性能
    print("Average Time to Score (s): {}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))

In [36]:
import numpy as np
import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape
n_train = train_data.shape[0]
train_features = all_features[:n_train].values
test_features = all_features[n_train:].values
train_labels = train_data.SalePrice.values.reshape(-1, 1)

In [37]:
# tree_params = {'max_depth':[None, 1, 3, 5, 7,10]}
# get_best_model_and_accuracy(model_RFR,tree_params,train_features,train_labels)

D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\model_selection\_validation.py:681: DataCon

Best Accuracy: -0.02164715760203597
Best Parameters: {'max_depth': None}
Average Time to Fit (s): 0.899
Average Time to Score (s): 0.008


In [38]:
model_RFR.fit(train_features,train_labels)
result = model_RFR.predict(test_features)

C:\Users\wss\AppData\Local\Temp/ipykernel_28680/54192094.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_RFR.fit(train_features,train_labels)


In [41]:
test_data['SalePrice'] = pd.Series(result)
submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
submission.to_csv('submission.csv', index=False)

array([127440.66, 154730.5 , 181318.03, ..., 152345.  , 112422.  ,
       224918.13])